# Часть 2. Соединение и предобработка скачанных отзывов 

In [1]:
import pandas as pd
import numpy as np

In [2]:
#увеличиваю отображение таблиц в ячейках вывода, чтобы показывало все данные
#pd.options.display.max_rows = 2250
#pd.options.display.max_columns = 100

## 2.1 подгружаю ранее скачанные отзывы с площадок

In [2]:
df_salyt_g= pd.read_csv('D:/Документы/Python/Площадки/google_review_салют.csv' )
df_salyt_g["location"] = 'Парк Горького. Салют'

In [3]:
df_piramidy_g= pd.read_csv('D:/Документы/Python/Площадки/google_review_piramidy2.csv' )
df_piramidy_g["location"] = 'Бунинские луга. Пирамиды'

In [4]:
df_salyt_yt= pd.read_csv('D:/Документы/Python/jupyter/df_salyt_youtube.csv' )#предобработанные

In [5]:
df_s_p_ya= pd.read_csv('df_salyt_piramidy_ya.csv' )#предобработанные

## Объединю отзывы из google карт

In [6]:
df_google = pd.concat([df_salyt_g, df_piramidy_g],ignore_index=True, axis=0).rename(columns={'rating':'date','duration':'comment'})
df_google

,name,date,review,comment,location
0,Дмитрий,9 месяцев назад,5 звезд,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют
1,Михаил Лось,11 месяцев назад,5 звезд,Здесь очень весело!),Парк Горького. Салют
2,Vika Andreeva,год назад,5 звезд,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют
3,Светлана Рабинович,год назад,5 звезд,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют
4,Игорь Тухкин,год назад,5 звезд,NaN,Парк Горького. Салют
...,...,...,...,...,...
1047,Максим Суманеев,3 года назад,5 звезд,NaN,Бунинские луга. Пирамиды
1048,Анатолий Булгаков,2 года назад,5 звезд,NaN,Бунинские луга. Пирамиды
1049,Лида Васильева,3 года назад,3 звезды,NaN,Бунинские луга. Пирамиды
1050,Rabbit_turbo,3 года назад,5 звезд,NaN,Бунинские луга. Пирамиды


## Преобразования таблицы

In [27]:
df_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      1052 non-null   object
 1   date      1052 non-null   object
 2   review    1052 non-null   object
 3   comment   369 non-null    object
 4   location  1052 non-null   object
dtypes: object(5)
memory usage: 41.2+ KB


In [7]:
df_google['review'] = df_google['review'].str.split().str[0]
df_google = df_google.astype({'review': np.int8})
df_google

,name,date,review,comment,location
0,Дмитрий,9 месяцев назад,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют
1,Михаил Лось,11 месяцев назад,5,Здесь очень весело!),Парк Горького. Салют
2,Vika Andreeva,год назад,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют
3,Светлана Рабинович,год назад,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют
4,Игорь Тухкин,год назад,5,NaN,Парк Горького. Салют
...,...,...,...,...,...
1047,Максим Суманеев,3 года назад,5,NaN,Бунинские луга. Пирамиды
1048,Анатолий Булгаков,2 года назад,5,NaN,Бунинские луга. Пирамиды
1049,Лида Васильева,3 года назад,3,NaN,Бунинские луга. Пирамиды
1050,Rabbit_turbo,3 года назад,5,NaN,Бунинские луга. Пирамиды


In [52]:
df_google['date'].value_counts().reset_index(name='Amount')

,index,Amount
0,3 года назад,363
1,год назад,263
2,2 года назад,229
3,4 года назад,185
4,10 месяцев назад,5
5,5 лет назад,4
6,9 месяцев назад,2
7,11 месяцев назад,1


In [8]:
df_google = df_google.replace({'9 месяцев назад': '2022', '11 месяцев назад': '2022', '10 месяцев назад': '2022', 'год назад': '2021', '3 года назад': '2019', '2 года назад':'2020', '4 года назад':'2018', '5 лет назад':'2017'})

In [9]:
df_google["resource"] = 'google'


In [10]:
df_playground = pd.concat([df_google,df_salyt_yt,df_s_p_ya ],ignore_index=True, axis=0)

In [11]:
df_playground = df_playground.drop(columns = ['Unnamed: 0'],axis = 1)

## 2.2 Определю и разделю имена на мужчин и женщин

####  c помощью генератора  имен (https://www.fakenamegenerator.com/order.php) создам базу данных с которой буду сравнивать имена в датасете

In [12]:
fake_name_ru= pd.read_csv('D:/Документы/Python/Площадки/FakeNameGenerator.com_09efa2a5.csv' )
fake_name_ru_en= pd.read_csv('D:/Документы/Python/Площадки/FakeNameGenerator.com_nnk4dfb59c5.csv')
fake_name_add= pd.read_csv('D:/Документы/Python/Площадки/FakeNameGenerator.com_add.csv' )
fake_name_ru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100073 entries, 0 to 100072
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Gender     100073 non-null  object
 1   GivenName  100073 non-null  object
 2   Surname    100073 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [16]:
#fake_name_ru2 = set(s for s in fake_name_ru_en[fake_name_ru_en['Gender'] == 'male']['Surname'] if s[-3:]== 'aya')

In [17]:
#fake_name_ru2

{'Pokrovskaya', 'Uspenskaya'}

In [13]:
fake_names = pd.concat([fake_name_ru, fake_name_ru_en,fake_name_add],ignore_index=True, axis=0)

In [14]:
male_names = set(fake_names[fake_names['Gender'] == 'male']['GivenName'].str.lower())
male_surnames = set(s[-3:] for s in fake_names[fake_names['Gender'] == 'male']['Surname'])


female_names= set(fake_names[fake_names['Gender'] == 'female']['GivenName'].str.lower())
female_surnames = set(s[-3:] for s in fake_names[fake_names['Gender'] == 'female']['Surname'])


In [15]:
# ошибочно попали окончания женских фамилий в мужские
drop_error = ['eva', 'ova', 'ina', 'aya' ]

In [16]:
for i in drop_error:
    male_surnames.remove(i)

In [17]:
df_playground2 = df_playground.copy()

In [18]:
df_playground2['name'] = df_playground['name'].str.split(' ').str[:2]
df_playground2

,name,date,review,comment,location,resource
0,[Дмитрий],2022,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют,google
1,"[Михаил, Лось]",2022,5,Здесь очень весело!),Парк Горького. Салют,google
2,"[Vika, Andreeva]",2021,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют,google
3,"[Светлана, Рабинович]",2021,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют,google
4,"[Игорь, Тухкин]",2021,5,NaN,Парк Горького. Салют,google
...,...,...,...,...,...,...
1946,"[Рафаэль, Т.]",2020,NaN,Интересная площадка для детей,Бунинские луга. Пирамиды,yandex
1947,"[Татьяна, Б]",2020,NaN,Очень хорошая площадка! Нам очень нравится. Го...,Бунинские луга. Пирамиды,yandex
1948,"[Ольга, Кузнецова]",2020,NaN,Мой ребенок готов сюда эмигрировать.,Бунинские луга. Пирамиды,yandex
1949,[Иван],2020,NaN,Шикарная площадка с водой. Дети часами могут п...,Бунинские луга. Пирамиды,yandex


### так как встречаются имена и фамилии не на своих местах, сначала нужно отсортировать, для удобства применения функций

In [19]:
def sort_names(names):
    #names = names.split('')
    name = np.NaN
    surname = np.NaN
    if len(names) > 1:
       
        if str(names[0]).lower() in female_names or str(names[0]).lower() in male_names:
            name = str(names[0]).lower()
        else:
            surname = str(names[0]).lower()
    
        if str(names[1]).lower() in female_names or str(names[1]).lower() in male_names:
            name = str(names[1]).lower()
        else:
            surname = str(names[1]).lower()
    if len(names) <= 1:
        if str(names[0]).lower() in female_names or str(names[0]).lower() in male_names:
            name = str(names[0]).lower()
        else:
            surname = str(names[0]).lower()
        
    else:
        pass
    
    return name, surname

In [20]:
def gender(names):
    score = np.NaN
    name, surname = sort_names(names)
    
    if str(name).isalpha() == True and len(str(name)) >1:
        
        if name in female_names:
                score = True

        if name in male_names:
                score = False
               
    if str(surname).isalpha() == True and len(str(surname))>1:
        
        if str(surname)[-3:] in female_surnames:
                score = True

        if str(surname)[-3:] in male_surnames:
                score = False
    
    else:
        pass             
    return score   
 

In [21]:
df_playground2['gender'] = df_playground2.apply(lambda x: gender(x['name']), axis =  1)
#df_playground2['gender2'] = df_playground2.apply(lambda x: gender(x['surname']), axis =  1)
df_playground2

,name,date,review,comment,location,resource,gender
0,[Дмитрий],2022,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют,google,False
1,"[Михаил, Лось]",2022,5,Здесь очень весело!),Парк Горького. Салют,google,False
2,"[Vika, Andreeva]",2021,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют,google,True
3,"[Светлана, Рабинович]",2021,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют,google,True
4,"[Игорь, Тухкин]",2021,5,NaN,Парк Горького. Салют,google,False
...,...,...,...,...,...,...,...
1946,"[Рафаэль, Т.]",2020,NaN,Интересная площадка для детей,Бунинские луга. Пирамиды,yandex,False
1947,"[Татьяна, Б]",2020,NaN,Очень хорошая площадка! Нам очень нравится. Го...,Бунинские луга. Пирамиды,yandex,True
1948,"[Ольга, Кузнецова]",2020,NaN,Мой ребенок готов сюда эмигрировать.,Бунинские луга. Пирамиды,yandex,True
1949,[Иван],2020,NaN,Шикарная площадка с водой. Дети часами могут п...,Бунинские луга. Пирамиды,yandex,False


In [37]:
df_playground2['gender'].value_counts(dropna = False).reset_index(name='Amount') # остались ники которые невозможно распределить

,index,Amount
0,False,870
1,True,727
2,NaN,354


In [22]:
df_playground2['name'] = df_playground['name']

In [23]:
df_playground2 = df_playground2.reindex(columns=['name', 'gender', 'date', 'review', 'comment', 'location', 'resource'])

In [86]:
df_playground2.to_csv('playground_reviews.csv', index=False)

In [82]:
df_playground2.to_excel (r'playground_reviews.xlsx')